## Logit lens

Trying logit lens

In [1]:
import numpy as numpy
import plotly.express as px
import torch

import circuitsvis as cv

import transformer_lens.utils as utils
from transformer_lens.hook_points import HookPoint
from transformer_lens import HookedTransformer, FactoredMatrix

import tqdm.auto as tqdm
from functools import partial

In [2]:
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

In [3]:
def vis_attn_patterns(model, text, layers, compact=True):
    str_tokens = model.to_str_tokens(text)
    logits, cache = model.run_with_cache(text, remove_batch_dim=True)

    if compact:
        for layer in layers:
            attention_pattern = cache["pattern", layer]
            display(cv.attention.attention_patterns(tokens=str_tokens, attention=attention_pattern))
    
    else:
        for layer in layers:
            attention_pattern = cache["pattern", layer]
            display(cv.attention.attention_heads(tokens=str_tokens, attention=attention_pattern))

In [4]:
model = HookedTransformer.from_pretrained("gpt2-small")

Loaded pretrained model gpt2-small into HookedTransformer


In [13]:
text = "The capital of Norway is"
answer = " Oslo"

logits, cache = model.run_with_cache(text)

In [31]:
acc_resid_stack = cache.accumulated_resid()
print(acc_resid_stack.shape)
scaled_resid_stack = cache.apply_ln_to_stack(acc_resid_stack)
print(scaled_resid_stack.shape)
Unembedding_matrix = model.W_U
print(Unembedding_matrix.shape)
print(scaled_resid_stack[:,0,0,:].shape)
logit_lens_final_logits = torch.mul(scaled_resid_stack[:, 0, ::], Unembedding_matrix)

torch.Size([13, 1, 6, 768])
torch.Size([13, 1, 6, 768])
torch.Size([768, 50257])
torch.Size([13, 768])


RuntimeError: The size of tensor a (768) must match the size of tensor b (50257) at non-singleton dimension 2

In [ ]:
logit_lens_results = torch.zeros([13, 6])
for layer in range(len(scaled_resid_stack[0])):
    for position in range(len(logits[0][0])):
        logit_lens_results = model.(scaled_resid_stack[layer][0][position])

TypeError: can only concatenate str (not "Tensor") to str